In [ ]:
import os
from google.colab import drive

# Step 1: Set Up Google Drive in Colab
drive.mount('/content/drive')

# Create directories in Google Drive for PDF and Excel files
pdf_folder_path = '/content/drive/MyDrive/output/pdf/'
#excel_folder_path = '/content/drive/MyDrive/output/excel/'
vector_folder_path = '/content/drive/MyDrive/output/newvectorstores/'
os.makedirs(pdf_folder_path, exist_ok=True)
#os.makedirs(excel_folder_path, exist_ok=True)
os.makedirs(vector_folder_path, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Step 2: Install Required Packages
!pip install -U langchain==0.1
!pip install -U langchain_openai
!pip install -U openai
!pip install -U ragas
!pip install -U pymupdf  # PyMuPDF for reading PDFs
!pip install -U chromadb
!pip install -U tiktoken
!pip install -U accelerate
!pip install -U bitsandbytes
!pip install -U datasets
!pip install -U sentence_transformers
!pip install -U FlagEmbedding
!pip install -U ninja
!pip install -U flash_attn --no-build-isolation
!pip install -U tqdm
!pip install -U rank_bm25
!pip install -U transformers
!pip install -U openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 7.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 93.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import shutil
import fitz  # PyMuPDF for PDF handling
from tqdm import tqdm
from langchain_community.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# Google Drive paths
pdf_folder_path = '/content/drive/MyDrive/output/pdf/'
# vector_folder_path = '/content/drive/MyDrive/output/vectorstores/'
# CHROMA_PATH = vector_folder_path + "chroma"
vector_folder_path = '/content/drive/MyDrive/output/newvectorstores/'
CHROMA_PATH = vector_folder_path + "newchroma"


# Function to load PDFs
def load_pdfs(folder_path):
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
    documents = []
    for pdf_file in tqdm(pdf_files, desc="Processing PDFs"):
        pdf_path = os.path.join(folder_path, pdf_file)
        try:
            doc = fitz.open(pdf_path)
            full_text = ""
            for page in doc:
                full_text += page.get_text()
            documents.append(Document(page_content=full_text, metadata={'title': pdf_file}))
        finally:
            doc.close()
    return documents

# Function to split text from documents
def split_text(documents: list[Document]):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=300,
        length_function=len,
        add_start_index=True,
    )
    chunks = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

    # Print length of each chunk, and sample content
    for i, chunk in enumerate(chunks[:5]):  # Print details for first 5 chunks for brevity
        print(f"Chunk {i+1}: Length {len(chunk.page_content)} characters, starts at index {chunk.metadata.get('start_index')}")
    return chunks

# Function to save data to Chroma
def save_to_chroma(chunks: list[Document]):
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)
    os.makedirs(CHROMA_PATH, exist_ok=True)

    # Initialize HuggingFace BGE Embeddings
    model_name = "BAAI/bge-large-en-v1.5"
    encode_kwargs = {'normalize_embeddings': True}
    hf_bge_embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_name,
        model_kwargs={'device': 'cuda'},
        encode_kwargs=encode_kwargs
    )

    db = Chroma.from_documents(
        chunks, hf_bge_embeddings, persist_directory=CHROMA_PATH
    )
    db.persist()
    print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}. Total number of characters processed: {sum(len(chunk.page_content) for chunk in chunks)}")

# Processing PDFs
pdf_documents = load_pdfs(pdf_folder_path)
pdf_chunks = split_text(pdf_documents)

# Combine all chunks for storage
all_chunks = pdf_chunks
save_to_chroma(all_chunks)





Processing PDFs: 100%|██████████| 46/46 [03:34<00:00,  4.67s/it]


Split 46 documents into 1170 chunks.
Chunk 1: Length 1974 characters, starts at index 0
Chunk 2: Length 1978 characters, starts at index 1712
Chunk 3: Length 1948 characters, starts at index 3401
Chunk 4: Length 1965 characters, starts at index 5088
Chunk 5: Length 1991 characters, starts at index 6778


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Saved 1170 chunks to /content/drive/MyDrive/output/newvectorstores/newchroma. Total number of characters processed: 2266000


In [ ]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


# Configuration for the model to be used
model_name = "llmware/dragon-deci-7b-v0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer,
                             config={"temperature": 1e-3, "do_sample": True,
                                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.eos_token_id})

questions_and_contexts = {
    "What is the evaluation purpose in this study?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ],
    "What is the research aim in this study?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ],
    "By which commercial Cloud provider(s) are the evaluated services supplied?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ],
    "What commercial Cloud computing services were evaluated in this study?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ],
    "What feature(s) was/were evaluated in this study?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ],
    "What components were concerned for the evaluated feature?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ],
    "What detailed configuration(s) was/were made in this study?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ]
}


# List of paper titles
titles = [
    "Performance Evaluation of Cloud Computing Offerings",
    "Evaluating Cloud Platform Architecture with the CARE Framework",
    "Cloud Computing for Comparative Genomics",
    "Cloudstone: Multi-Platform, Multi-Language Benchmark and Measurement Tools for Web 2.0",
    "Cloud Computing for Parallel Scientific HPC Applications: Feasibility of Running Coupled Atmosphere-Ocean Climate Models on Amazon's EC2",
    "C-Meter_A_Framework_for_Performance_Analysis_of_Computing_Clouds.pdf",
    "HPC on Competitive Cloud Resources",
    "Web Server Farm in the Cloud: Performance Evaluation and Dynamic Architecture",
    "Abstractions for Loosely-Coupled and Ensemble-based Simulations on Azure",
    "Cost-effective HPC: The Community or the Cloud?",
    "Performance Analysis of High Performance Computing Applications on the Amazon Web Services Cloud",
    "A Performance Analysis of EC2 Cloud Computing Services for Scientific Computing",
    "Using Clouds for Metagenomics: A Case Study",
    "Amazon S3 for Science Grids: A Viable Solution?",
    "Metabolic Flux Analysis in the Cloud",
    "Scientific Workflow Applications on Amazon EC2",
    "A Quantitative Analysis of High Performance Computing with Amazon's EC2 Infrastructure: The Death of the Local Cluster?",
    "Evaluating Caching and Storage Options on the Amazon Web Services Cloud",
    "Avoiding Performance Fluctuation in Cloud Storage",
    "Evaluating the Cost-Benefit of Using Cloud Computing to Extend the Capacity of Clusters",
    "Case Study for Running HPC Applications in Public Clouds",
    "Early Observations on the Performance of Windows Azure",
    "Response Time for Cloud Computing Providers",
    "EC2 Performance Analysis for Resource Provisioning of Service-Oriented Application",
    "CloudCmp: Comparing Public Cloud Providers",
    "The Impact of Virtualization on Network Performance of Amazon EC2 Data Center",
    "Cost-Benefit Analysis of Cloud Computing versus Desktop Grids",
    "eScience in the Cloud: A MODIS Satellite Data Reprojection and Reduction Pipeline in the Windows Azure Platform",
    "Commodity Grid Computing with Amazon's S3 and EC2",
    "Benchmarking Amazon EC2 for High-Performance Scientific Computing",
    "Scientific Computing in the Cloud",
    "Empirical Evaluation of Latency-sensitive Application Performance in the Cloud",
    "Performance and Cost Assessment of Cloud Services",
    "Scientific Computing using Virtual High-Performance Computing: A Case Study using the Amazon Elastic Computing Cloud",
    "The Cost of Doing Science on the Cloud: The Montage Example",
    "Data Sharing Options for Scientific Workflows on Amazon EC2",
    "AzureBlast: A Case Study of Developing Science Application on the Cloud",
    "Building a Database on S3",
    "An Evaluation of Alternative Architectures for Transaction Processing in the Cloud",
    "An Evaluation of Amazon's Grid Computing Services: EC2, S3 and SQS",
    "On the Performance Variability of Production Cloud Services",
    "Can Cloud Computing Reach the TOP500?",
    "Performance Measurement of a Private Cloud in the OpenCirrus Testbed",
    "Runtime Measurements in the Cloud: Observing, Analyzing, and Reducing Variance",
    "HPC Benchmarks on Amazon EC2",
    "Storage Access Optimization with Virtual Machine Migration and Basic Performance Analysis of Amazon EC2"
]

# Initialize BGE Embeddings
model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True}
hf_bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

vector_folder_path = '/content/drive/MyDrive/output/newvectorstores/'
CHROMA_PATH = vector_folder_path + "newchroma"
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=hf_bge_embeddings)

# Prepare to collect all data
data = {question: [] for question in questions_and_contexts}

for title in tqdm(titles, desc="Processing titles"):
    for question, contexts in questions_and_contexts.items():
        # Format the context with the title just once
        context_to_use = contexts[0].format(title=title) + question
        # Form the prompt without redundant title information
        prompt = f"Context: {context_to_use}\n\n"

        # Use Chroma to find the most relevant contexts
        results = db.similarity_search_with_relevance_scores(prompt, k=1)
        if results:
            # Generate response using the language model with only the necessary prompt
            response = text_gen_pipeline(prompt, max_length=512, truncation=True)[0]['generated_text']
            response = response.split('\n\n')[1] if '\n\n' in response else response  # Assuming response starts after the first new line
        else:
            fallback_prompt = f"Unable to find relevant context. Proceeding with base knowledge.\n{prompt}"
            # Generate response using the language model with base knowledge
            response = text_gen_pipeline(fallback_prompt, max_length=512, truncation=True)[0]['generated_text']
            response = response.split('\n\n')[1] if '\n\n' in response else response  # Adjust based on your model's output format

        # Append the response for the current question to the respective list in the dictionary
        data[question].append(response.strip())

# Create a DataFrame
df = pd.DataFrame(data, index=titles)  # Use titles as the index

# Save to Excel
output_path = "/content/drive/MyDrive/output/research_paper_queries_corrected3new.xlsx"
df.to_excel(output_path)

print(f"Excel file has been created at {output_path} with the responses.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

Processing titles:   0%|          | 0/46 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/backends/cuda/__init__.py:342: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, with updated signature.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Processing titles:   2%|▏         | 1/46 [01:28<1:06:27, 88.60s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` t

Excel file has been created at /content/drive/MyDrive/output/research_paper_queries_corrected3new.xlsx with the responses.


In [ ]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


# Configuration for the model to be used
model_name = "llmware/dragon-deci-7b-v0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer,
                             config={"temperature": 1e-3, "do_sample": True,
                                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.eos_token_id})

questions_and_contexts = {
    "Discuss the primary objective of the study": [
        "Discuss the primary objective of the study detailed in '{title}'. What gaps in cloud computing research does this study aim to address, and how does it propose to advance our understanding or application of this technology?"
    ],
    "Analyze the main research goals": [
        "Analyze the main research goals in '{title}'. How do these goals contribute to the evolution of cloud computing technologies, and what new perspectives or innovations does the study introduce to the field?"
    ],
    "Critically assess the commercial cloud providers discussed": [
        "Critically assess the commercial cloud providers discussed in '{title}', detailing the specific services that are evaluated. Explain how these services are relevant to the study’s objectives and what criteria are used to evaluate their effectiveness."
    ],
    "Provide an in-depth analysis of the cloud computing services evaluated": [
        "Provide an in-depth analysis of the cloud computing services evaluated in '{title}', focusing on the evaluation criteria and methodologies used. Compare these services in terms of their performance, scalability, and reliability. What conclusions does the study draw about the comparative effectiveness of these services?"
    ],
    "Examine the key features of cloud computing that are critically assessed": [
        "Examine the key features of cloud computing that are critically assessed in '{title}'. Discuss how these features influence the efficiency and security of cloud infrastructures. What are the study's recommendations for enhancing these features?"
    ]
}


# List of paper titles
titles = [
    "Performance Evaluation of Cloud Computing Offerings",
    "Evaluating Cloud Platform Architecture with the CARE Framework",
    "Cloud Computing for Comparative Genomics",
    "Cloudstone: Multi-Platform, Multi-Language Benchmark and Measurement Tools for Web 2.0",
    "Cloud Computing for Parallel Scientific HPC Applications: Feasibility of Running Coupled Atmosphere-Ocean Climate Models on Amazon's EC2",
    "C-Meter_A_Framework_for_Performance_Analysis_of_Computing_Clouds.pdf",
    "HPC on Competitive Cloud Resources",
    "Web Server Farm in the Cloud: Performance Evaluation and Dynamic Architecture",
    "Abstractions for Loosely-Coupled and Ensemble-based Simulations on Azure",
    "Cost-effective HPC: The Community or the Cloud?",
    "Performance Analysis of High Performance Computing Applications on the Amazon Web Services Cloud",
    "A Performance Analysis of EC2 Cloud Computing Services for Scientific Computing",
    "Using Clouds for Metagenomics: A Case Study",
    "Amazon S3 for Science Grids: A Viable Solution?",
    "Metabolic Flux Analysis in the Cloud",
    "Scientific Workflow Applications on Amazon EC2",
    "A Quantitative Analysis of High Performance Computing with Amazon's EC2 Infrastructure: The Death of the Local Cluster?",
    "Evaluating Caching and Storage Options on the Amazon Web Services Cloud",
    "Avoiding Performance Fluctuation in Cloud Storage",
    "Evaluating the Cost-Benefit of Using Cloud Computing to Extend the Capacity of Clusters",
    "Case Study for Running HPC Applications in Public Clouds",
    "Early Observations on the Performance of Windows Azure",
    "Response Time for Cloud Computing Providers",
    "EC2 Performance Analysis for Resource Provisioning of Service-Oriented Application",
    "CloudCmp: Comparing Public Cloud Providers",
    "The Impact of Virtualization on Network Performance of Amazon EC2 Data Center",
    "Cost-Benefit Analysis of Cloud Computing versus Desktop Grids",
    "eScience in the Cloud: A MODIS Satellite Data Reprojection and Reduction Pipeline in the Windows Azure Platform",
    "Commodity Grid Computing with Amazon's S3 and EC2",
    "Benchmarking Amazon EC2 for High-Performance Scientific Computing",
    "Scientific Computing in the Cloud",
    "Empirical Evaluation of Latency-sensitive Application Performance in the Cloud",
    "Performance and Cost Assessment of Cloud Services",
    "Scientific Computing using Virtual High-Performance Computing: A Case Study using the Amazon Elastic Computing Cloud",
    "The Cost of Doing Science on the Cloud: The Montage Example",
    "Data Sharing Options for Scientific Workflows on Amazon EC2",
    "AzureBlast: A Case Study of Developing Science Application on the Cloud",
    "Building a Database on S3",
    "An Evaluation of Alternative Architectures for Transaction Processing in the Cloud",
    "An Evaluation of Amazon's Grid Computing Services: EC2, S3 and SQS",
    "On the Performance Variability of Production Cloud Services",
    "Can Cloud Computing Reach the TOP500?",
    "Performance Measurement of a Private Cloud in the OpenCirrus Testbed",
    "Runtime Measurements in the Cloud: Observing, Analyzing, and Reducing Variance",
    "HPC Benchmarks on Amazon EC2",
    "Storage Access Optimization with Virtual Machine Migration and Basic Performance Analysis of Amazon EC2"
]

# Initialize BGE Embeddings
model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True}
hf_bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs=encode_kwargs
)

vector_folder_path = '/content/drive/MyDrive/output/newvectorstores/'
CHROMA_PATH = vector_folder_path + "newchroma"
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=hf_bge_embeddings)

# Prepare to collect all data
data = {question: [] for question in questions_and_contexts}

for title in tqdm(titles, desc="Processing titles"):
    for question, contexts in questions_and_contexts.items():
        # Format the context with the title just once
        context_to_use = contexts[0].format(title=title) + question
        # Form the prompt without redundant title information
        prompt = f"Context: {context_to_use}\n\n"

        # Use Chroma to find the most relevant contexts
        results = db.similarity_search_with_relevance_scores(prompt, k=1)
        if results:
            # Generate response using the language model with only the necessary prompt
            response = text_gen_pipeline(prompt, max_length=512, truncation=True)[0]['generated_text']
            response = response.split('\n\n')[1] if '\n\n' in response else response  # Assuming response starts after the first new line
        else:
            fallback_prompt = f"Unable to find relevant context. Proceeding with base knowledge.\n{prompt}"
            # Generate response using the language model with base knowledge
            response = text_gen_pipeline(fallback_prompt, max_length=512, truncation=True)[0]['generated_text']
            response = response.split('\n\n')[1] if '\n\n' in response else response  # Adjust based on your model's output format

        # Append the response for the current question to the respective list in the dictionary
        data[question].append(response.strip())

# Create a DataFrame
df = pd.DataFrame(data, index=titles)  # Use titles as the index

# Save to Excel
output_path = "/content/drive/MyDrive/output/research_paper_queries_corrected3new.xlsx"
df.to_excel(output_path)

print(f"Excel file has been created at {output_path} with the responses.")


tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

...

transformers_v4_35_2__modeling_llama.py:   0%|          | 0.00/56.6k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/llmware/dragon-deci-7b-v0:
- transformers_v4_35_2__modeling_llama.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/llmware/dragon-deci-7b-v0:
- modeling_decilm.py
- transformers_v4_35_2__modeling_attn_mask_utils.py
- transformers_v4_35_2__modeling_llama.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


pytorch_model.bin:   0%|          | 0.00/14.1G [00:00<?, ?B/s]

Processing titles:   0%|          | 0/46 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/backends/cuda/__init__.py:342: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, with updated signature.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Processing titles:   2%|▏         | 1/46 [01:13<54:51, 73.15s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to 

Excel file has been created at /content/drive/MyDrive/output/research_paper_queries_corrected3new.xlsx with the responses.


In [ ]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


# Configuration for the model to be used
model_name = "llmware/dragon-deci-7b-v0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer,
                             config={"temperature": 1e-3, "do_sample": True,
                                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.eos_token_id})

questions_and_contexts = {
    "What is the purpose of the study in this document?": [
        "Please summarize the primary objective of the study outlined in the document titled '{title}'. What specific aspect of cloud computing does it aim to evaluate or improve?"
    ],
    "Identify the main research goal presented in this document.": [
        "Identify the main research goal presented in the document'{title}'. What advancements or contributions to cloud computing does this document propose?"
    ],
    "Which commercial cloud providers are evaluated in this document?": [
        "List the commercial cloud providers evaluated in '{title}'. Which services from these providers are analyzed, and why are they significant to the study's goals?"
    ],
    "What cloud computing services are evaluated in this study?": [
        "Detail the specific cloud computing services examined in the study '{title}'. What are the criteria for evaluation, and how do these services compare in terms of performance and scalability?"
    ],
    "What features are evaluated in this document?": [
        "What key features of cloud computing are assessed in '{title}'? How do these features impact the overall efficiency and security of cloud services?"
    ],
    "What components are concerned for the evaluated feature?": [
        "Discuss the critical components analyzed in '{title}' related to the evaluated feature. How do these components influence the outcomes of the study?"
    ],
    "What configurations were made in this study?": [
        "Explain the detailed configurations made during the experiments in '{title}'. Include information on hardware specifications, software settings, and any special network arrangements."
    ]
}



# List of paper titles
titles = [
    "Performance Evaluation of Cloud Computing Offerings",
    "Evaluating Cloud Platform Architecture with the CARE Framework",
    "Cloud Computing for Comparative Genomics",
    "Cloudstone: Multi-Platform, Multi-Language Benchmark and Measurement Tools for Web 2.0",
    "Cloud Computing for Parallel Scientific HPC Applications: Feasibility of Running Coupled Atmosphere-Ocean Climate Models on Amazon's EC2",
    "C-Meter_A_Framework_for_Performance_Analysis_of_Computing_Clouds.pdf",
    # "HPC on Competitive Cloud Resources",
    # "Web Server Farm in the Cloud: Performance Evaluation and Dynamic Architecture",
    # "Abstractions for Loosely-Coupled and Ensemble-based Simulations on Azure",
    # "Cost-effective HPC: The Community or the Cloud?",
    # "Performance Analysis of High Performance Computing Applications on the Amazon Web Services Cloud",
    # "A Performance Analysis of EC2 Cloud Computing Services for Scientific Computing",
    # "Using Clouds for Metagenomics: A Case Study",
    # "Amazon S3 for Science Grids: A Viable Solution?",
    # "Metabolic Flux Analysis in the Cloud",
    # "Scientific Workflow Applications on Amazon EC2",
    # "A Quantitative Analysis of High Performance Computing with Amazon's EC2 Infrastructure: The Death of the Local Cluster?",
    # "Evaluating Caching and Storage Options on the Amazon Web Services Cloud",
    # "Avoiding Performance Fluctuation in Cloud Storage",
    # "Evaluating the Cost-Benefit of Using Cloud Computing to Extend the Capacity of Clusters",
    # "Case Study for Running HPC Applications in Public Clouds",
    # "Early Observations on the Performance of Windows Azure",
    # "Response Time for Cloud Computing Providers",
    # "EC2 Performance Analysis for Resource Provisioning of Service-Oriented Application",
    # "CloudCmp: Comparing Public Cloud Providers",
    # "The Impact of Virtualization on Network Performance of Amazon EC2 Data Center",
    # "Cost-Benefit Analysis of Cloud Computing versus Desktop Grids",
    # "eScience in the Cloud: A MODIS Satellite Data Reprojection and Reduction Pipeline in the Windows Azure Platform",
    # "Commodity Grid Computing with Amazon's S3 and EC2",
    # "Benchmarking Amazon EC2 for High-Performance Scientific Computing",
    # "Scientific Computing in the Cloud",
    # "Empirical Evaluation of Latency-sensitive Application Performance in the Cloud",
    # "Performance and Cost Assessment of Cloud Services",
    # "Scientific Computing using Virtual High-Performance Computing: A Case Study using the Amazon Elastic Computing Cloud",
    # "The Cost of Doing Science on the Cloud: The Montage Example",
    # "Data Sharing Options for Scientific Workflows on Amazon EC2",
    # "AzureBlast: A Case Study of Developing Science Application on the Cloud",
    # "Building a Database on S3",
    # "An Evaluation of Alternative Architectures for Transaction Processing in the Cloud",
    # "An Evaluation of Amazon's Grid Computing Services: EC2, S3 and SQS",
    # "On the Performance Variability of Production Cloud Services",
    # "Can Cloud Computing Reach the TOP500?",
    # "Performance Measurement of a Private Cloud in the OpenCirrus Testbed",
    # "Runtime Measurements in the Cloud: Observing, Analyzing, and Reducing Variance",
    # "HPC Benchmarks on Amazon EC2",
    # "Storage Access Optimization with Virtual Machine Migration and Basic Performance Analysis of Amazon EC2"
]

# Initialize BGE Embeddings
model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True}
hf_bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs=encode_kwargs
)

vector_folder_path = '/content/drive/MyDrive/output/vectorstores/'
CHROMA_PATH = vector_folder_path + "chroma"
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=hf_bge_embeddings)

# Prepare to collect all data
data = {question: [] for question in questions_and_contexts}

for title in tqdm(titles, desc="Processing titles"):
    for question, contexts in questions_and_contexts.items():
        # Format the context with the title just once
        context_to_use = contexts[0].format(title=title) + question
        # Form the prompt without redundant title information
        prompt = f"Context: {context_to_use}\n\n"

        # Use Chroma to find the most relevant contexts
        results = db.similarity_search_with_relevance_scores(prompt, k=1)
        if results:
            # Generate response using the language model with only the necessary prompt
            response = text_gen_pipeline(prompt, max_length=512, truncation=True)[0]['generated_text']
            response = response.split('\n\n')[1] if '\n\n' in response else response  # Assuming response starts after the first new line
        else:
            fallback_prompt = f"Unable to find relevant context. Proceeding with base knowledge.\n{prompt}"
            # Generate response using the language model with base knowledge
            response = text_gen_pipeline(fallback_prompt, max_length=512, truncation=True)[0]['generated_text']
            response = response.split('\n\n')[1] if '\n\n' in response else response  # Adjust based on your model's output format

        # Append the response for the current question to the respective list in the dictionary
        data[question].append(response.strip())

# Create a DataFrame
df = pd.DataFrame(data, index=titles)  # Use titles as the index

# Save to Excel
output_path = "/content/drive/MyDrive/output/research_paper_queries_corrected3.xlsx"
df.to_excel(output_path)

print(f"Excel file has been created at {output_path} with the responses.")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing titles:   0%|          | 0/6 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/backends/cuda/__init__.py:3

Excel file has been created at /content/drive/MyDrive/output/research_paper_queries_corrected3.xlsx with the responses.


In [ ]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


# Configuration for the model to be used
model_name = "llmware/dragon-deci-7b-v0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer,
                             config={"temperature": 1e-3, "do_sample": True,
                                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.eos_token_id})

questions_and_contexts = {
    "What is the evaluation purpose in this study?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ],
    "What is the research aim in this study?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ],
    "By which commercial Cloud provider(s) are the evaluated services supplied?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ],
    "What commercial Cloud computing services were evaluated in this study?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ],
    "What feature(s) was/were evaluated in this study?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ],
    "What components were concerned for the evaluated feature?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ],
    "What detailed configuration(s) was/were made in this study?": [
        "You are an expert in cloud performance engineering. Now by reading the paper titled '{title}', please answer: "
    ]
}


# List of paper titles
titles = [
    "Performance Evaluation of Cloud Computing Offerings",
    "Evaluating Cloud Platform Architecture with the CARE Framework",
    "Cloud Computing for Comparative Genomics",
    "Cloudstone: Multi-Platform, Multi-Language Benchmark and Measurement Tools for Web 2.0",
    "Cloud Computing for Parallel Scientific HPC Applications: Feasibility of Running Coupled Atmosphere-Ocean Climate Models on Amazon's EC2",
    "C-Meter_A_Framework_for_Performance_Analysis_of_Computing_Clouds.pdf",
    "HPC on Competitive Cloud Resources",
    "Web Server Farm in the Cloud: Performance Evaluation and Dynamic Architecture",
    "Abstractions for Loosely-Coupled and Ensemble-based Simulations on Azure",
    "Cost-effective HPC: The Community or the Cloud?",
    "Performance Analysis of High Performance Computing Applications on the Amazon Web Services Cloud",
    "A Performance Analysis of EC2 Cloud Computing Services for Scientific Computing",
    "Using Clouds for Metagenomics: A Case Study",
    "Amazon S3 for Science Grids: A Viable Solution?",
    "Metabolic Flux Analysis in the Cloud",
    "Scientific Workflow Applications on Amazon EC2",
    "A Quantitative Analysis of High Performance Computing with Amazon's EC2 Infrastructure: The Death of the Local Cluster?",
    "Evaluating Caching and Storage Options on the Amazon Web Services Cloud",
    "Avoiding Performance Fluctuation in Cloud Storage",
    "Evaluating the Cost-Benefit of Using Cloud Computing to Extend the Capacity of Clusters",
    "Case Study for Running HPC Applications in Public Clouds",
    "Early Observations on the Performance of Windows Azure",
    "Response Time for Cloud Computing Providers",
    "EC2 Performance Analysis for Resource Provisioning of Service-Oriented Application",
    "CloudCmp: Comparing Public Cloud Providers",
    "The Impact of Virtualization on Network Performance of Amazon EC2 Data Center",
    "Cost-Benefit Analysis of Cloud Computing versus Desktop Grids",
    "eScience in the Cloud: A MODIS Satellite Data Reprojection and Reduction Pipeline in the Windows Azure Platform",
    "Commodity Grid Computing with Amazon's S3 and EC2",
    "Benchmarking Amazon EC2 for High-Performance Scientific Computing",
    "Scientific Computing in the Cloud",
    "Empirical Evaluation of Latency-sensitive Application Performance in the Cloud",
    "Performance and Cost Assessment of Cloud Services",
    "Scientific Computing using Virtual High-Performance Computing: A Case Study using the Amazon Elastic Computing Cloud",
    "The Cost of Doing Science on the Cloud: The Montage Example",
    "Data Sharing Options for Scientific Workflows on Amazon EC2",
    "AzureBlast: A Case Study of Developing Science Application on the Cloud",
    "Building a Database on S3",
    "An Evaluation of Alternative Architectures for Transaction Processing in the Cloud",
    "An Evaluation of Amazon's Grid Computing Services: EC2, S3 and SQS",
    "On the Performance Variability of Production Cloud Services",
    "Can Cloud Computing Reach the TOP500?",
    "Performance Measurement of a Private Cloud in the OpenCirrus Testbed",
    "Runtime Measurements in the Cloud: Observing, Analyzing, and Reducing Variance",
    "HPC Benchmarks on Amazon EC2",
    "Storage Access Optimization with Virtual Machine Migration and Basic Performance Analysis of Amazon EC2"
]

# Initialize BGE Embeddings
model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True}
hf_bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

vector_folder_path = '/content/drive/MyDrive/output/vectorstores/'
CHROMA_PATH = vector_folder_path + "chroma"
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=hf_bge_embeddings)

# Prepare to collect all data
data = {question: [] for question in questions_and_contexts}

for title in tqdm(titles, desc="Processing titles"):
    for question, contexts in questions_and_contexts.items():
        # Format the context with the title just once
        context_to_use = contexts[0].format(title=title) + question
        # Form the prompt without redundant title information
        prompt = f"Context: {context_to_use}\n\n"


        # Use Chroma to find the most relevant contexts
        results = db.similarity_search_with_relevance_scores(prompt, k=1)
        if results:
            # Generate response using the language model with only the necessary prompt
            response = text_gen_pipeline(prompt, max_length=512, truncation=True)[0]['generated_text']
        else:
            fallback_prompt = f"Unable to find relevant context. Proceeding with base knowledge.\n{prompt}"
            # Generate response using the language model with base knowledge
            response = text_gen_pipeline(fallback_prompt, max_length=512, truncation=True)[0]['generated_text']

        # Append the response for the current question to the respective list in the dictionary
        data[question].append(response.strip())

# Create a DataFrame
df = pd.DataFrame(data, index=titles)  # Use titles as the index

# Save to Excel
output_path = "/content/drive/MyDrive/output/research_paper_queries_corrected1.xlsx"
df.to_excel(output_path)

print(f"Excel file has been created at {output_path} with the responses.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing titles:   0%|          | 0/46 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/backends/cuda/__init__.py:

Excel file has been created at /content/drive/MyDrive/output/research_paper_queries_corrected1.xlsx with the responses.


In [ ]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


# Configuration for the model to be used
model_name = "llmware/dragon-deci-7b-v0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer,
                             config={"temperature": 1e-3, "do_sample": True,
                                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.eos_token_id})

# Questions and their respective contexts
questions_and_contexts = {
    "What is the evaluation purpose in this study?": [
        "You are an expert in the cloud performance engineering now by reading this paper answer the question,keyword evaluation"
    ],
    "What is the research aim in this study?": [
      "You are an expert in the cloud performance engineering now by reading this paper answer the question, specific aim like monitoring computing"
    ],
    "By which commercial Cloud provider(s) are the evaluated services supplied?": [
     "You are an expert in the cloud performance engineering now by reading this paper answer the question, names of cloud service providers discussed in the paper"
    ],
    "What commercial Cloud computing services were evaluated in this study?": [
        "You are an expert in the cloud performance engineering now by reading this paper answer the question check types of cloud services discussed"
    ],
    "What feature(s) was/were evaluated in this study?": [
     "You are an expert in the cloud performance engineering now by reading this paper answer the question, check the list of features,perfomance evaluated in the study"
    ],
    "What components were concerned for the evaluated feature?": [
       "You are an expert in the cloud performance engineering now by reading this paper answer the question"
    ],
    "What detailed configuration(s) was/were made in this study?": [
      "You are an expert in the cloud performance engineering now by reading this paper answer the question, like architecture or any methods"
    ]
}

# List of paper titles
titles = [
    "Performance Evaluation of Cloud Computing Offerings",
    "Evaluating Cloud Platform Architecture with the CARE Framework",
    "Cloud Computing for Comparative Genomics",
    "Cloudstone: Multi-Platform, Multi-Language Benchmark and Measurement Tools for Web 2.0",
    "Cloud Computing for Parallel Scientific HPC Applications: Feasibility of Running Coupled Atmosphere-Ocean Climate Models on Amazon's EC2",
    "C-Meter_A_Framework_for_Performance_Analysis_of_Computing_Clouds.pdf",
    "HPC on Competitive Cloud Resources",
    "Web Server Farm in the Cloud: Performance Evaluation and Dynamic Architecture",
    "Abstractions for Loosely-Coupled and Ensemble-based Simulations on Azure",
    "Cost-effective HPC: The Community or the Cloud?",
    "Performance Analysis of High Performance Computing Applications on the Amazon Web Services Cloud",
    "A Performance Analysis of EC2 Cloud Computing Services for Scientific Computing",
    "Using Clouds for Metagenomics: A Case Study",
    "Amazon S3 for Science Grids: A Viable Solution?",
    "Metabolic Flux Analysis in the Cloud",
    "Scientific Workflow Applications on Amazon EC2",
    "A Quantitative Analysis of High Performance Computing with Amazon's EC2 Infrastructure: The Death of the Local Cluster?",
    "Evaluating Caching and Storage Options on the Amazon Web Services Cloud",
    "Avoiding Performance Fluctuation in Cloud Storage",
    "Evaluating the Cost-Benefit of Using Cloud Computing to Extend the Capacity of Clusters",
    "Case Study for Running HPC Applications in Public Clouds",
    "Early Observations on the Performance of Windows Azure",
    "Response Time for Cloud Computing Providers",
    "EC2 Performance Analysis for Resource Provisioning of Service-Oriented Application",
    "CloudCmp: Comparing Public Cloud Providers",
    "The Impact of Virtualization on Network Performance of Amazon EC2 Data Center",
    "Cost-Benefit Analysis of Cloud Computing versus Desktop Grids",
    "eScience in the Cloud: A MODIS Satellite Data Reprojection and Reduction Pipeline in the Windows Azure Platform",
    "Commodity Grid Computing with Amazon's S3 and EC2",
    "Benchmarking Amazon EC2 for High-Performance Scientific Computing",
    "Scientific Computing in the Cloud",
    "Empirical Evaluation of Latency-sensitive Application Performance in the Cloud",
    "Performance and Cost Assessment of Cloud Services",
    "Scientific Computing using Virtual High-Performance Computing: A Case Study using the Amazon Elastic Computing Cloud",
    "The Cost of Doing Science on the Cloud: The Montage Example",
    "Data Sharing Options for Scientific Workflows on Amazon EC2",
    "AzureBlast: A Case Study of Developing Science Application on the Cloud",
    "Building a Database on S3",
    "An Evaluation of Alternative Architectures for Transaction Processing in the Cloud",
    "An Evaluation of Amazon's Grid Computing Services: EC2, S3 and SQS",
    "On the Performance Variability of Production Cloud Services",
    "Can Cloud Computing Reach the TOP500?",
    "Performance Measurement of a Private Cloud in the OpenCirrus Testbed",
    "Runtime Measurements in the Cloud: Observing, Analyzing, and Reducing Variance",
    "HPC Benchmarks on Amazon EC2",
    "Storage Access Optimization with Virtual Machine Migration and Basic Performance Analysis of Amazon EC2"
]

# Initialize BGE Embeddings
model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True}
hf_bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

vector_folder_path = '/content/drive/MyDrive/output/vectorstores/'
CHROMA_PATH = vector_folder_path + "chroma"
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=hf_bge_embeddings)

# Prepare to collect all data
data = {question: [] for question in questions_and_contexts}

# Process each title with each context
for title in tqdm(titles, desc="Processing titles"):
    for question, contexts in questions_and_contexts.items():
        # Combine contexts into a single string
        context_to_use = " ".join(contexts)
        # Format the prompt to include the paper title, question, and context
        prompt = f"Title: {title}\nContext: {context_to_use}\nQuestion: {question}\n\n"

        # Use Chroma to find the most relevant contexts
        results = db.similarity_search_with_relevance_scores(prompt, k=1)
        if results:
            # Generate response using the language model with only the necessary prompt
            response = text_gen_pipeline(prompt, max_length=512, truncation=True)[0]['generated_text']
        else:
            fallback_prompt = f"Unable to find relevant context. Proceeding with base knowledge.\n{prompt}"
            # Generate response using the language model with base knowledge
            response = text_gen_pipeline(fallback_prompt, max_length=512, truncation=True)[0]['generated_text']

        # Append the response for the current question to the respective list in the dictionary
        data[question].append(response.strip())

# Create a DataFrame
df = pd.DataFrame(data, index=titles)  # Use titles as the index

# Save to Excel
output_path = "/content/drive/MyDrive/output/research_paper_queries_corrected.xlsx"
df.to_excel(output_path)

print(f"Excel file has been created at {output_path} with the responses.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Processing titles:   0%|          | 0/46 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/backends/cuda/__init__.py:

Excel file has been created at /content/drive/MyDrive/output/research_paper_queries_corrected.xlsx with the responses.


In [ ]:
import pandas as pd
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


# Configuration for the model to be used
model_name = "llmware/dragon-deci-7b-v0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)
text_gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer,
                             config={"temperature": 1e-3, "do_sample": True,
                                     "eos_token_id": tokenizer.eos_token_id, "pad_token_id": tokenizer.eos_token_id})

# Questions and their respective contexts
questions_and_contexts = {
    "What is the evaluation purpose in this study?": [
        "Focus on evaluating the efficiency and cost-effectiveness of cloud services designed for handling large datasets.",
        "Experiments conducted using real-world data applications to assess performance under various loads."
    ],
    "What is the research aim in this study?": [
        "Investigate how adaptable cloud computing platforms are for diverse scientific research needs.",
        "Study includes surveys and technical tests to evaluate flexibility and scalability."
    ],
    "By which commercial Cloud provider(s) are the evaluated services supplied?": [
        "Detailed review of major cloud providers including AWS, Azure, and Google Cloud.",
        "Evaluation based on enterprise needs such as security, scalability, and support services."
    ],
    "What commercial Cloud computing services were evaluated in this study?": [
        "This study compares Infrastructure-as-a-Service (IaaS) and Platform-as-a-Service (PaaS) offerings from several key market players.",
        "Focus on performance, cost, and configuration flexibility."
    ],
    "What feature(s) was/were evaluated in this study?": [
        "Evaluation of security protocols and features in cloud environments that support multiple tenants.",
        "Specific attention to data isolation, encryption practices, and access controls."
    ],
    "What components were concerned for the evaluated feature?": [
        "Study focuses on the components of cloud storage solutions critical for ensuring high availability.",
        "Tests include redundancy mechanisms, data retrieval speeds, and failover protocols."
    ],
    "What detailed configuration(s) was/were made in this study?": [
        "Configuration details include hardware specs, software versions, and network settings used during testing.",
        "Analysis of different setup configurations to determine optimal settings for performance."
    ]
}

# List of paper titles
titles = [
    "Performance Evaluation of Cloud Computing Offerings",
    "Evaluating Cloud Platform Architecture with the CARE Framework",
    "Cloud Computing for Comparative Genomics",
    "Cloudstone: Multi-Platform, Multi-Language Benchmark and Measurement Tools for Web 2.0",
    "Cloud Computing for Parallel Scientific HPC Applications: Feasibility of Running Coupled Atmosphere-Ocean Climate Models on Amazon's EC2",
    "C-Meter_A_Framework_for_Performance_Analysis_of_Computing_Clouds.pdf",
    "HPC on Competitive Cloud Resources",
    "Web Server Farm in the Cloud: Performance Evaluation and Dynamic Architecture",
    "Abstractions for Loosely-Coupled and Ensemble-based Simulations on Azure",
    "Cost-effective HPC: The Community or the Cloud?",
    "Performance Analysis of High Performance Computing Applications on the Amazon Web Services Cloud",
    "A Performance Analysis of EC2 Cloud Computing Services for Scientific Computing",
    "Using Clouds for Metagenomics: A Case Study",
    "Amazon S3 for Science Grids: A Viable Solution?",
    "Metabolic Flux Analysis in the Cloud",
    "Scientific Workflow Applications on Amazon EC2",
    "A Quantitative Analysis of High Performance Computing with Amazon's EC2 Infrastructure: The Death of the Local Cluster?",
    "Evaluating Caching and Storage Options on the Amazon Web Services Cloud",
    "Avoiding Performance Fluctuation in Cloud Storage",
    "Evaluating the Cost-Benefit of Using Cloud Computing to Extend the Capacity of Clusters",
    "Case Study for Running HPC Applications in Public Clouds",
    "Early Observations on the Performance of Windows Azure",
    "Response Time for Cloud Computing Providers",
    "EC2 Performance Analysis for Resource Provisioning of Service-Oriented Application",
    "CloudCmp: Comparing Public Cloud Providers",
    "The Impact of Virtualization on Network Performance of Amazon EC2 Data Center",
    "Cost-Benefit Analysis of Cloud Computing versus Desktop Grids",
    "eScience in the Cloud: A MODIS Satellite Data Reprojection and Reduction Pipeline in the Windows Azure Platform",
    "Commodity Grid Computing with Amazon's S3 and EC2",
    "Benchmarking Amazon EC2 for High-Performance Scientific Computing",
    "Scientific Computing in the Cloud",
    "Empirical Evaluation of Latency-sensitive Application Performance in the Cloud",
    "Performance and Cost Assessment of Cloud Services",
    "Scientific Computing using Virtual High-Performance Computing: A Case Study using the Amazon Elastic Computing Cloud",
    "The Cost of Doing Science on the Cloud: The Montage Example",
    "Data Sharing Options for Scientific Workflows on Amazon EC2",
    "AzureBlast: A Case Study of Developing Science Application on the Cloud",
    "Building a Database on S3",
    "An Evaluation of Alternative Architectures for Transaction Processing in the Cloud",
    "An Evaluation of Amazon's Grid Computing Services: EC2, S3 and SQS",
    "On the Performance Variability of Production Cloud Services",
    "Can Cloud Computing Reach the TOP500?",
    "Performance Measurement of a Private Cloud in the OpenCirrus Testbed",
    "Runtime Measurements in the Cloud: Observing, Analyzing, and Reducing Variance",
    "HPC Benchmarks on Amazon EC2",
    "Storage Access Optimization with Virtual Machine Migration and Basic Performance Analysis of Amazon EC2"
]

# Initialize BGE Embeddings
model_name = "BAAI/bge-large-en-v1.5"
encode_kwargs = {'normalize_embeddings': True}
hf_bge_embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

vector_folder_path = '/content/drive/MyDrive/output/vectorstores/'
CHROMA_PATH = vector_folder_path + "chroma"
db = Chroma(persist_directory=CHROMA_PATH, embedding_function=hf_bge_embeddings)

# Prepare to collect all data
data = {question: [] for question in questions_and_contexts}

# Process each title with each context
for title in tqdm(titles, desc="Processing titles"):
    for question, contexts in questions_and_contexts.items():
        # Combine contexts into a single string
        context_to_use = " ".join(contexts)
        # Use Chroma to find the most relevant contexts
        results = db.similarity_search_with_relevance_scores(f"{context_to_use} {question}", k=1)
        if results:
            best_document, _score = results[0]
            prompt = best_document.page_content + f"\n\n### Question: {question}"
            # Generate response using the language model
            response = text_gen_pipeline(prompt, max_length=512)[0]['generated_text']
        else:
            prompt = f"Unable to find relevant context. Proceeding with base knowledge.\n\n### Question: {question}"
            # Generate response using the language model with base knowledge
            response = text_gen_pipeline(prompt, max_length=512)[0]['generated_text']
        data[question].append(response)

# Create a DataFrame
df = pd.DataFrame(data, index=titles)

# Save to Excel
df.to_excel("research_paper_queries.xlsx")

print("Excel file has been created with the responses.")
